In [1]:
import psycopg2
import pandas as pd


In [2]:
DB_CONFIG = {
    'host': 'localhost',
    'port': 5432,
    'user': 'auser',
    'password': 'Practica1_g4',
    'dbname': 'postgres'
}
DB_ANOMALIAS = 'postgres'
CSV_PATH = ['/Users/fernandocalderon05/Escritorio2/UIDE/MAESTRIA/4. INTELIGENCIA DE NEGOCIOS/PRACTICA 1/Practica1_G4/IN_P1_G4/dataset/population_data.csv']
BATCH_SIZE = 10
MAX_WORKERS = 10

In [3]:
def verificar_conexion():
    try:
        with psycopg2.connect(**DB_CONFIG) as conn:
            with conn.cursor() as cur:
                cur.execute("SELECT 1;")
                _ = cur.fetchone()
        print("✅ Conexión exitosa a PostgreSQL.")
    except Exception as e:
        print(f"❌ Error al conectar: {e}")

In [4]:
def crear_tabla(column_names, tabla='population'):
    columns_str = ', '.join([f'"{col}" VARCHAR(100)' for col in column_names])
    sql = f"CREATE TABLE IF NOT EXISTS {tabla} (id SERIAL PRIMARY KEY,{columns_str});"
    try:
        with psycopg2.connect(**DB_CONFIG) as conn:
            with conn.cursor() as cur:
                cur.execute(sql)
                conn.commit()
        print("✅ Tabla creada.")
    except Exception as e:
        print(f"❌ Error al crear la tabla: {e}")

In [5]:
def consulta_a_dataframe(query, params=None):
    try:
        with psycopg2.connect(**DB_CONFIG) as conn:
            df = pd.read_sql_query(query, conn, params=params)
        print("✅ Consulta ejecutada con éxito.")
        return df
    except Exception as e:
        print(f"❌ Error al ejecutar consulta: {e}")
        return pd.DataFrame()  # retorna vacío si falla

In [6]:
def insertar_registros(data, tabla='population'):
    try:
        # Columnas entre comillas dobles si tienen espacios o mayúsculas
        columnas = ', '.join([f'"{col}"' for col in data.columns])
        placeholders = ', '.join(['%s'] * len(data.columns))
        sql = f"INSERT INTO {tabla} ({columnas}) VALUES ({placeholders});"
        with psycopg2.connect(**DB_CONFIG) as conn:
            with conn.cursor() as cur:
                for row in data.itertuples(index=False, name=None):
                    cur.execute(sql, row)
            conn.commit()

        print(f"✅ Insertados {len(data)} registros.")

    except Exception as e:
        print(f"❌ Error al insertar dato: {e}")

In [13]:
def main():
    try:
        verificar_conexion()

        print("⏳ Leyendo archivo CSV...")
        for csv_path in CSV_PATH:
            df = pd.read_csv(csv_path, delimiter=',', encoding='utf-8')
            column_names = df.columns.tolist()
            crear_tabla(column_names)
            insertar_registros(df)
        print("⏳ Insertando registros en la base de datos...")
    except Exception as e:
        print(f"❌ Error en el proceso principal: {e}")

if __name__ == '__main__':
    main()

✅ Conexión exitosa a PostgreSQL.
⏳ Leyendo archivo CSV...
✅ Tabla creada.
✅ Insertados 264 registros.
✅ Consulta ejecutada con éxito.
⏳ Insertando registros en la base de datos...


/var/folders/pd/0xzmj7j51p7997xsvfv18wsh0000gn/T/ipykernel_9990/1920335490.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=params)


In [14]:
## pueden crear varios df en base a la consulta que deseen hacer por ejemplo: df1
df1 = consulta_a_dataframe("SELECT * FROM population LIMIT 5;")
df1.drop(columns=['Unnamed: 62'], inplace=True)
df.head()

✅ Consulta ejecutada con éxito.


/var/folders/pd/0xzmj7j51p7997xsvfv18wsh0000gn/T/ipykernel_9990/1920335490.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn, params=params)


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,105264.0,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,...,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,35530081.0,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,...,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0,29784193.0,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,2873457.0,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0,76965.0,NaN
